In [1]:
from bs4 import BeautifulSoup
import requests
import csv



headers = {
	'Accept' : '*/*',
	'Accept-Language': 'en-US,en;q=0.5',
	'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82',
}


This will take all the names of councils in England & Wales from the online website
In the future, Scottish councils should also be added

In [2]:
councils_url = "https://www.local.gov.uk/our-support/guidance-and-resources/communications-support/digital-councils/social-media/go-further/a-z-councils-online"
councils_content = requests.get(councils_url,headers=headers).text
councils_soup = BeautifulSoup(councils_content,'html.parser')
search = councils_soup.find_all(rel="noreferrer noopener")
council_names = [link.text for link in search]


After each council name, the following keywords are added. These oculd potentially be amended to improve performance.

In [3]:
grants       = [council + " home grants" for council in council_names]
renewables   = [council + " renewables grants" for council in council_names]
green        = [council + " green grants" for council in council_names]
funding      = [council + " home funding" for council in council_names]
loans        = [council + " home loans" for council in council_names]
search_terms = grants + renewables + green + funding + loans
print(search_terms)

['Adur and Worthing Borough Council home grants', 'Adur District Council home grants', 'Allerdale Borough Council home grants', 'Amber Valley Borough Council home grants', 'Arun District Council home grants', 'Ashfield District Council home grants', 'Ashford Borough Council home grants', 'Aylesbury Vale District Council home grants', 'Babergh District Council home grants', 'Barnsley Metropolitan Borough Council home grants', 'Barrow-in-Furness Borough Council home grants', 'Basildon Borough Council home grants', 'Basingstoke and Deane Borough Council home grants', 'Bassetlaw District Council home grants', 'Bath and North East Somerset Council home grants', 'Bedford Borough Council home grants', 'Birmingham City Council home grants', 'Blaby District Council home grants', 'Blackburn with Darwen Borough Council home grants', 'Blackpool Borough Council home grants', 'Blaenau Gwent County Borough Council home grants', 'Bolsover District Council home grants', 'Bolton Metropolitan Borough Cou

The following part of the code takes quite a while to run.
Each search term is put through a google search and the first link is scraped.
The results are then de-duped.

In [4]:
search_results_urls = []

for search in search_terms:
    url = 'https://www.google.com/search'
    parameters = {'q': search}
    content = requests.get(url, headers=headers, params=parameters).text
    soup = BeautifulSoup(content, 'html.parser')
    search = soup.find(id='search')
    first_link = search.find('a')

    if first_link:
        search_results_urls.append(first_link['href'])

# Print all the resulting URLs
for url in search_results_urls:
    print(url)

https://www.adur-worthing.gov.uk/housing/adaptations-and-grants/
https://www.adur-worthing.gov.uk/housing/adaptations-and-grants/
https://www.allerdale.gov.uk/en/housing/housing-grants/
https://www.solid-wall-grant.co.uk/#:~:text=The%20grant%20provided%20through%20Amber,will%20my%20property%20look%20like%3F
https://www.arun.gov.uk/news-archive/local-authority-delivery-scheme-available-to-help-lower-energy-bills-3210/#:~:text=Eligible%20residents%20in%20the%20Arun,Solar%20PV%20panels
https://www.ashfield.gov.uk/housing/repairs-improvements/adapting-and-improving-your-home/affordable-warmth-grant/
https://www.ashford.gov.uk/housing/grants/private-sector-grants/
https://energysavinggenie.co.uk/aylesbury-vale-district-council-flexible-energy-la-flex/
https://www.babergh.gov.uk/loans-grants-and-assistance
https://www.barnsley.gov.uk/services/housing/energy-at-home/make-your-home-warmer-and-cheaper-to-run/
https://www.barrowbc.gov.uk/residents/private-sector-housing/housing-grants-and-improv

In [7]:
print(len(search_results_urls))

1890


In [5]:
dictionary = {url: name for url, name in zip(search_results_urls,council_names+council_names+council_names+council_names+council_names)}

In [6]:
print(len(dictionary))
# Create and write the URLs to a CSV file
csv_filename = 'search_results.csv'
# Open the CSV file for writing
with open(csv_filename, 'w', newline='') as csv_file:
    # Create a CSV writer
    csv_writer = csv.writer(csv_file)

    # Write the header row
    csv_writer.writerow(['URL', 'Council Name'])

    # Write the data from the deduplicated dictionary
    for council, url in dictionary.items():
        csv_writer.writerow([council, url])


1438
